## Part 1
Given your actual map, after `10000` bursts of activity, **how many bursts cause a node to become infected**? Do not count nodes that begin infected.

### Grid class

In [1]:
class Grid(object):
    def __init__(self, map_):
        self._grid = {}
        i, j_start = -(len(map_) // 2), -(len(map_[0]) // 2)
        for row in map_:
            j = j_start
            for c in row:
                self.set_node(c == '#', (i, j))
                j += 1
            i += 1
        self._direction = (-1, 0)  # Up
        self._position = (0, 0)
        self._bursts, self._got_infected = 0, 0

    def get_node(self, position=None):
        if position is None:
            position = self._position
        if position not in self._grid:
            self._grid[position] = False
        return self._grid[position]

    def set_node(self, infected, position=None):
        if position is None:
            position = self._position
        self._grid[position] = infected

    DIR_STR = {(-1, 0): '↑', (0, 1): '>', (1, 0): '↓', (0, -1): '<'}
    TURNS_CW = ((-1, 0), (0, 1), (1, 0), (0, -1))

    def __str__(self):
        i_coords = set([pos[0] for pos in self._grid.keys()])
        j_coords = set([pos[1] for pos in self._grid.keys()])
        si, ei, sj, ej = min(i_coords), max(i_coords), min(j_coords), max(
            j_coords)
        pprint = []
        for i in range(si, ei + 1):
            row = []
            for j in range(sj, ej + 1):
                pos = (i, j)
                show_arrow = pos == self._position
                d_char = Grid.DIR_STR[self._direction] if show_arrow else ' '
                char = '#' if self.get_node(pos) else '.'
                row.append(f'{d_char}{char}{d_char}')
            pprint.append(''.join(row))
        return f'After burst #{self._bursts:02}, {self._got_infected} got infected.\n' + '\n'.join(pprint)

    def burst(self, iterations=1, debug=False):
        total_dirs = len(Grid.TURNS_CW)  # Total number of directions = 4
        for i in range(iterations):            
            # Turn clockwise if infected, counterclockwise otherwise
            turn_direction = 1 if self.get_node() else -1
            next_dir_index = Grid.TURNS_CW.index(self._direction) + total_dirs + turn_direction
            self._direction = Grid.TURNS_CW[next_dir_index % total_dirs]
            if not self.get_node():
                self._got_infected += 1
            self.set_node(not self.get_node())
            self._position = (self._position[0] + self._direction[0], self._position[1] + self._direction[1])
            self.get_node(self._position)
            self._bursts += 1
            if debug:
                print(self)

### Testing

In [2]:
test_in = '..#/#../...'.split('/')
grid = Grid(test_in)
grid.burst(7)
print(grid)
assert grid._got_infected == 5, f'Expected got infected = 5, got {grid._got_infected}'
grid.burst(70-7)
print(grid)
assert grid._got_infected == 41, f'Expected got infected = 41, got {grid._got_infected}'

After burst #07, 5 got infected.
 #  . >.> # 
 #  #  #  . 
 .  .  .  . 
After burst #70, 41 got infected.
 .  .  .  #  #  .  . 
 .  .  #  .  .  #  . 
 .  #  .  .  .  .  # 
 #  .  # ↑.↑ .  .  # 
 #  .  #  .  .  #  . 
 .  .  .  #  #  .  . 


### Processing puzzle input

In [3]:
puzzle_in = [line[:-1] for line in open('in/day22.txt', 'r')]
grid = Grid(puzzle_in)
grid.burst(10000)
print(f'Part 1 answer is: {grid._got_infected} got infected after 10K bursts')

Part 1 answer is: 5433 got infected after 10K bursts
